In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd

import plotly.express as px

import dash
from dash import Dash, dcc, html
from dash.dependencies import Output, Input

import dash_bootstrap_components as dbc
import plotly.graph_objects as go

### Governs Scrapped 

In [3]:
governs = ['القاهرة', 'الأسكندرية', 'اسيوط', 'أسوان', 'الجيزة', 'بني سويف', 'دمنهور', 'دمياط', 'الفيوم', 'حلوان', 'إسماعيلية', 'الأكسر', 'الغربية', 'بور سعيد',
      'شرم الشيخ', 'سهاج', 'شبرا الخيمة', 'السويس', 'المحلة الكبرى', 'المنصورة', 'طنطا', 'الزقازيق', 'الخصوص', 'المنيا', 'الأقصر', 'قنا', 'سوهاج', 'شبين الكوم', 'الغردقة',
      'بنها', 'كفر الشيخ', 'ملوي', 'العريش', 'بلبيس', 'مدينة العاشر من رمضان', 'مرسى مطروح', 'ميت غمر', 'كفر الدوار', 'قليوب', 'دسوق', 'ابو كبير', 'جرجا', 'أخميم',
      
      'المطرية', 'إدكو', 'بلقاس', 'زفتى', 'سمالوط', 'منوف', 'السنبلاوين', 'طهطا', '(مركز ناصر (بوش', 'أشمون', 'منفلوط', 'سنورس', 'بني مزار', 'فاقوس', 'طلخا', 'أرمنت',
      'مغاغة', 'ديروط', 'كوم امبو', 'كفر الزيات', 'أبو تيج', 'إدفو', 'رشيد', 'إسنا', 'دكرنس', 'أبنوب', 'طما', 'بيلا', 'القناطر', 'الفشن', 'المنشي', 'الجمالية', 'فُوّه'
      , 'منيا القمح', 'الخارجة', 'قوص', 'الخانكة', 'أبو قرقاص', 'ببا', 'سمنود', 'شبين القناطر', 'إبشواي', 'شربين', 'ديرب نجم', 'بسيون', 'دشنا', 'الحامول', 'فرشوط', 'تلا'
      
      , 'شباس الشهداء', 'طامية', 'مشتول السوق', 'مدينه السادات', 'الغنايم', 'إطسا', 'البلينا', 'حوش عيسى', 'مطاي', 'حهينة', 'سيدي سالم', 'نجع حمادي', 'قويسنا'
      , 'ههيا', 'أبو المطامير', 'العبور', 'البداري', 'القنايات', 'طوخ', 'الباجور', 'إيتاى البارود', 'دير مواس', 'بلطيم', 'أبو حماد', 'أبو حمص', 'نبروة', 'دراو'
      , 'سمسطا الوقف', 'الواسطى', 'كوم حمادة', 'القصير', 'قلين', 'المنوفية', 'فارسكور', 'رأس غارب', 'دار السلام', 'ميت سلسبيل', 'الحسينية', 'كفر البطيخ'
      
      , 'القنطره', 'شبراخيت', 'المحمودية', 'الوقف', 'قها', 'قطور', 'كفر سعد', 'قفط']

### Function That Scraps Prayers Time

In [4]:
from datetime import datetime as dt
from datetime import timedelta

import json

In [5]:
with open('govenLinkData.json' , 'r') as file:
    htmlLinks = json.load(file)
    
#"https://www.timesprayer.com/prayer-times-in-"+htmlLinks['الأسكندرية']

In [6]:
from bs4 import BeautifulSoup
from lxml import etree
import requests

In [7]:
def getLastThird(location):
    link = htmlLinks[location]
    
    URL = "https://www.timesprayer.com/prayer-times-in-" + link
    webpage = requests.get(URL)
    soup = BeautifulSoup(webpage.content, "html.parser")
    dom = etree.HTML(str(soup))
    
    
    fajrTime = dom.xpath('//*[@id="bodysection"]/div[4]/div[1]/div[1]/ul/li[1]/text()')[0].strip()
    duhrTime = dom.xpath('//*[@id="bodysection"]/div[4]/div[1]/div[1]/ul/li[3]/text()')[0].strip()
    aasrTime = dom.xpath('//*[@id="bodysection"]/div[4]/div[1]/div[1]/ul/li[4]/text()')[0].strip()
    maghTime = dom.xpath('//*[@id="bodysection"]/div[4]/div[1]/div[1]/ul/li[5]/text()')[0].strip()
    ishaTime = dom.xpath('//*[@id="bodysection"]/div[4]/div[1]/div[1]/ul/li[6]/text()')[0].strip()

    
    
    #print('Maghrib is at: ', maghTime)
    #print('Fajr is at: '   , fajrTime)

    t_maghrib = pd.to_datetime(maghTime)
    t_fajr    = pd.to_datetime(fajrTime)

    totalDifference = t_fajr - t_maghrib
    lastThird = (totalDifference + timedelta(days = 1)) / 3 

    startOfLastThird = (t_fajr - lastThird).time()
    #stringFormat = startOfLastThird.strftime("%H:%M:%S")
    stringFormat = startOfLastThird.strftime("%H:%M") + ' AM'
    
    #print("The Last Third of The Night Starts at :", stringFormat)
    
    all_times = {
        'fajr' : fajrTime,
        'duhr' : duhrTime,
        'aasr' : aasrTime,
        'magh' : maghTime,
        'isha' : ishaTime,
        'thrd' : stringFormat
    }
    
    return all_times

In [8]:
getLastThird('الأسكندرية')

{'fajr': '3:19 AM',
 'duhr': '11:57 AM',
 'aasr': '3:36 PM',
 'magh': '6:53 PM',
 'isha': '8:24 PM',
 'thrd': '00:30 AM'}

### Layout Design

In [9]:
# Empty Space to Center The Dropdown Cell
empty_space = html.Div(
    children=[
        html.Div(
            style={
                "width": "20rem"
            },
        )
    ]
)

In [10]:
# Cards For Prayers Time
fajr_time = dbc.Card(
    dbc.Row(
        [
            html.H5(
                "Fajr", 
                style={
                    'textAlign': 'center',
                    'fontsize': '5em',
                }
            ),
            html.H5(
                getLastThird('الأسكندرية')['fajr'],
                id='fajr-time',
                style={
                    'textAlign': 'center',
                    'fontsize': '5em',
                }
            ),
        ]
    ),
    style={
        "width": "15rem"
    },
)


duhr_time = dbc.Card(
    dbc.Row(
        [
            html.H5(
                "Duhr", 
                style={
                    'textAlign': 'center',
                    'fontsize': '5em',
                }
            ),
            html.H5(
                getLastThird('الأسكندرية')['duhr'],
                id='duhr-time',
                style={
                    'textAlign': 'center',
                    'fontsize': '5em',
                }
            ),
        ]
    ),
    style={
        "width": "15rem"
    },
)

aasr_time = dbc.Card(
    dbc.Row(
        [
            html.H5(
                "Aasr", 
                style={
                    'textAlign': 'center',
                    'fontsize': '5em',
                }
            ),
            html.H5(
                getLastThird('الأسكندرية')['aasr'],
                id='aasr-time',
                style={
                    'textAlign': 'center',
                    'fontsize': '5em',
                }
            ),
        ]
    ),
    style={
        "width": "15rem"
    },
)

magh_time = dbc.Card(
    dbc.Row(
        [
            html.H5(
                "Maghrib", 
                style={
                    'textAlign': 'center',
                    'fontsize': '5em',
                }
            ),
            html.H5(
                getLastThird('الأسكندرية')['magh'],
                id='magh-time',
                style={
                    'textAlign': 'center',
                    'fontsize': '5em',
                }
            ),
        ]
    ),
    style={
        "width": "15rem"
    },
)

isha_time = dbc.Card(
    dbc.Row(
        [
            html.H5(
                "Isha", 
                style={
                    'textAlign': 'center',
                    'fontsize': '5em',
                }
            ),
            html.H5(
                getLastThird('الأسكندرية')['isha'],
                id='isha-time',
                style={
                    'textAlign': 'center',
                    'fontsize': '5em',
                }
            ),
        ]
    ),
    style={
        "width": "15rem"
    },
)


In [11]:
app = dash.Dash(
    external_stylesheets=[dbc.themes.BOOTSTRAP]
)

app.layout = html.Div(
    children = [
        
        # Title Header
        dbc.Alert(
            html.H1(
                "حاسبة الثلث الاخير من الليل",
            style = {
                'textAlign' : 'center',
                'color' : 'black'
            }),
            color="success"
        ),
        
        # MarkDown
        dbc.Row(
            [
                dbc.Col(
                    empty_space
                ),
        html.Div(
            [
                dcc.Dropdown(
                    options = governs,
                    value = 'الأسكندرية',
                    id='demo-dropdown'
                ),
            ],
            style={
                "width": "20rem"
            },
        ),
                dbc.Col(
                    empty_space
                ),
            ],
        ),
        
        html.Br(),
        
        # Prayers Time
        dbc.Row(
            [
                dbc.Col(
                    fajr_time
                ),
                dbc.Col(
                    duhr_time
                ),
                dbc.Col(
                    aasr_time
                ),
                dbc.Col(
                    magh_time
                ),
                dbc.Col(
                    isha_time
                ),
            ]
        ),
        
        html.Br(),
        
        # Last Third
        dbc.Alert(
            [html.H1(
                "الثلث الأخير يبدا اليوم من الساعة",
            style = {
                'textAlign' : 'center',
                'color' : 'black'
            }
            ),
            html.H4(
                getLastThird('الأسكندرية')['thrd'],
                id='thrd-time',
                style={
                    'textAlign': 'center',
                    'fontsize': '5em',
                }
            )],
            color="dark"
        ),   
        
        html.Br(),
        
        html.H5('يَنْزِلُ رَبُّنَا تَبَارَكَ وتَعَالَى كُلَّ لَيْلَةٍ إلى السَّمَاءِ الدُّنْيَا حِينَ يَبْقَى ثُلُثُ اللَّيْلِ الآخِرُ يقولُ: مَن يَدْعُونِي، فأسْتَجِيبَ له مَن يَسْأَلُنِي فَأُعْطِيَهُ، مَن يَسْتَغْفِرُنِي فأغْفِرَ له',
                style={
                    'textAlign': 'center',
                    'fontsize': '5em',
                }
               )
    ]
)


# Fajr Callback
@app.callback(
    Output(component_id='fajr-time',component_property='children'),
    Output(component_id='duhr-time',component_property='children'),
    Output(component_id='aasr-time',component_property='children'),
    Output(component_id='magh-time',component_property='children'),
    Output(component_id='isha-time',component_property='children'),
    Output(component_id='thrd-time',component_property='children'),
    Input(component_id='demo-dropdown',component_property='value'))
def update_time(dropdown_value):
    return tuple(getLastThird(dropdown_value).values())



In [12]:
if __name__ == "__main__":
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [21/May/2022 02:12:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2022 02:12:45] "POST /_dash-update-component HTTP/1.1" 200 -
